## 1 combine_data

In [1]:
import pandas as pd

In [2]:
train_path = 'C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\raw\\train.csv'
test_path = 'C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\raw\\test.csv'

In [3]:
train_df = pd.read_csv(train_path, index_col=0)
test_df = pd.read_csv(test_path, index_col=0)

In [4]:
train_df.shape, test_df.shape

((6261, 7), (2504, 6))

In [5]:
train_df["is_train"] = 1
test_df["is_train"] = 0

In [6]:
combined_df = pd.concat([train_df, test_df], sort=False)

In [7]:
combined_df.head(2)

,id,atm_group,address,address_rus,lat,long,target,is_train
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.01150,1
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.02971,1


In [8]:
combined_df.shape

(8765, 8)

In [9]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8765 entries, 0 to 2503
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8765 non-null   float64
 1   atm_group    8765 non-null   float64
 2   address      8765 non-null   object 
 3   address_rus  8345 non-null   object 
 4   lat          8345 non-null   float64
 5   long         8345 non-null   float64
 6   target       6261 non-null   float64
 7   is_train     8765 non-null   int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 616.3+ KB


In [10]:
combined_df.isnull().sum()

id                0
atm_group         0
address           0
address_rus     420
lat             420
long            420
target         2504
is_train          0
dtype: int64

## 2 find_gaps

In [11]:
combined_df[combined_df['address_rus'].isnull()]

,id,atm_group,address,address_rus,lat,long,target,is_train
40,4510.0,496.5,PER. BAZARNY 4 SPASSK-DALNY,NaN,NaN,NaN,0.039191,1
150,347.0,1022.0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,NaN,NaN,-0.145001,1
182,1591.0,1942.0,16A BEREJKOVSK NAB MOSCOW,NaN,NaN,NaN,0.080034,1
192,440.0,1942.0,113 MIRA AV. TOGLIATTI,NaN,NaN,NaN,-0.082956,1
214,556.0,1942.0,133A MOZHAYSKOE H/W. MOSCOW,NaN,NaN,NaN,0.061517,1
...,...,...,...,...,...,...,...,...
2346,3199.0,8083.0,"MOROZOVA P.L., 86 KHabarovsk",NaN,NaN,NaN,NaN,0
2399,5354.0,5478.0,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",NaN,NaN,NaN,NaN,0
2471,3333.0,8083.0,"OKTYABRSKAYA, 105 Tver",NaN,NaN,NaN,NaN,0
2474,2247.0,3185.5,"UL. TELEVIZORNAJA, KRASNOJARSK",NaN,NaN,NaN,NaN,0


## 3 data_enrichment_via_API

In [20]:
geocoded_df = pd.read_csv('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\external\\geocoded_data.csv')

In [23]:
geocoded_df[['source', 'result', 'qc', 'qc_complete', 'qc_house', 'qc_geo']]

,source,result,qc,qc_complete,qc_house,qc_geo
0,PER. BAZARNY 4 SPASSK-DALNY,"Приморский край, г Спасск-Дальний, пер Базарны...",1,9,2,4
1,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,"Респ Татарстан, г Набережные Челны",1,3,10,4
2,16A BEREJKOVSK NAB MOSCOW,г Москва,1,3,10,4
3,113 MIRA AV. TOGLIATTI,"Самарская обл, г Тольятти, ул Мира, влд 113",0,5,2,0
4,133A MOZHAYSKOE H/W. MOSCOW,"г Москва, ш Можайское, д 133А",1,9,10,2
...,...,...,...,...,...,...
415,"MOROZOVA P.L., 86 KHabarovsk","г Хабаровск, ул Морозова Павла Леонтьевича, д 86",0,5,2,0
416,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",г Красноярск,1,3,10,4
417,"OKTYABRSKAYA, 105 Tver",г Тверь,1,3,10,4
418,"UL. TELEVIZORNAJA, KRASNOJARSK","г Красноярск, ул Телевизорная",0,4,10,2


In [41]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
geocoded_df[geocoded_df['qc_geo'] < 3]

,Unnamed: 0,source,result,postal_code,country,...,qc_complete,qc_house,qc,unparsed_parts,metro
3,0,113 MIRA AV. TOGLIATTI,"Самарская обл, г Тольятти, ул Мира, влд 113",445054.0,Россия,...,5,2,0,NaN,NaN
4,0,133A MOZHAYSKOE H/W. MOSCOW,"г Москва, ш Можайское, д 133А",143026.0,Россия,...,9,10,1,ДОМ/В,NaN
5,0,63A IZMAYLOVSKIY AV. MOSCOW,"г Москва, ул Измайловский Вал, д 63А",105318.0,Россия,...,10,10,0,NaN,NaN
6,0,14/2 BUMAZHNIY AVE MOSCOW,"г Москва, проезд Бумажный, д 14 стр 2",127015.0,Россия,...,5,2,0,NaN,"[{'distance': 0.4, 'line': 'Большая кольцевая линия', 'name': 'Савёловская'}, {'distance': 0.5, 'line': 'Серпуховско-Тимирязевская', 'name': 'Савёловская'}, {'distance': 0.6, 'line': 'Белорусско-Савеловский', 'name': 'Савёловская'}]"
7,0,45 YANA RAYNISA ST. MOSCOW,"г Москва, б-р Яна Райниса, д 45",125373.0,Россия,...,10,10,0,NaN,"[{'distance': 1.9, 'line': 'Курско-Рижский', 'name': 'Трикотажная'}, {'distance': 2.1, 'line': 'Таганско-Краснопресненская', 'name': 'Сходненская'}, {'distance': 2.2, 'line': 'Таганско-Краснопресненская', 'name': 'Планерная'}]"
...,...,...,...,...,...,...,...,...,...,...,...
410,0,"D. 6, UL. KOMSOMOL'SKAYA YAROSLAVL G","г Ярославль, ул Комсомольская, д 6",150000.0,Россия,...,5,2,0,NaN,NaN
412,0,"D. 137 A, UL. MICHURINSKA TAMBOV G","г Тамбов, ул Мичуринская, д 137А",392032.0,Россия,...,5,2,0,NaN,NaN
414,0,STAROVATUTINSKIJ PR MOSKVA,"г Москва, проезд Староватутинский",NaN,Россия,...,4,10,0,NaN,NaN
415,0,"MOROZOVA P.L., 86 KHabarovsk","г Хабаровск, ул Морозова Павла Леонтьевича, д 86",680012.0,Россия,...,5,2,0,NaN,NaN
